<a href="https://colab.research.google.com/github/moosemorse/AI_Text_Detector/blob/main/TATG_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [14]:
#gets rid of installation dialogue
%%capture
!pip install transformers
!pip install pytorch
!pip install datasets
!pip install pytorch_lightning

In [15]:
!nvidia-smi

Mon Aug 28 18:02:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import os
import matplotlib.pyplot as plt
from google.colab import files, drive
from datasets import load_dataset
import pandas as pd
import seaborn as sns
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer
import numpy as np
import copy


In [17]:
#mount drive, gain access to file in google drive
drive.mount('/content/drive', force_remount=False)

#obtain csv file and store in var 'df' as dataframe
train_path = "drive/MyDrive/GPT-wiki-intro.csv"
df = pd.read_csv(train_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Inspect data

In [ ]:
#data to describe csv file
print(df.describe())

In [ ]:
df.head()

In [ ]:
print(df.iloc[:].loc[:, ['wiki_intro', 'generated_intro']])
#iloc dictates the rows indexed
#loc dictates the columns extracted

In [ ]:
#visualisation to compare data for human-written text and ai-written text
#testing seaborn and these could be helpful for evaluation afterwards
sns.countplot(x = 'wiki_intro_len', data = df)
plt.show()

sns.countplot(x = 'generated_intro_len', data = df)
plt.show()

In [ ]:
df.max()

In [ ]:
len(df)

# Dataset

to create a useful chatGPT dataset (which will be returning tensors):

In [18]:
#dataset class inherits dataset module imported from torch
class ChatGPT_Dataset(Dataset):

  def __init__(self, data_path, tokenizer, max_token_len = 512):
    self.data_path = data_path
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len
    self._prepare_data()

  #cleans dataframe to create dataset with text needed and labels
  #1 represents human written, 0 represents generated
  def _prepare_data(self):
    data = pd.read_csv(self.data_path)
    #dataframe for generated data, additional column label added
    generated = pd.DataFrame({'text': data['generated_intro'], 'label': 0})
    #dataframe for human-written data, additional column label added
    wiki = pd.DataFrame({'text': data['wiki_intro'], 'label': 1})
    #concatenate both dataframes
    self.data = pd.concat([generated, wiki])

  def __len__(self):
    return (len(self.data))

  def __getitem__(self, index):
    #find row in data which contains text and label
    item = self.data.iloc[index]
    message = str(item.text)
    label = torch.LongTensor([item['label']])
    #tokenize our message, returning tensors for input ids and an attention mask
    #truncation and padding used so all tensors are the same size
    tokens = self.tokenizer.encode_plus(message,
                                        add_special_tokens = True,
                                        return_tensors ='pt',
                                        truncation = True,
                                        max_length = self.max_token_len,
                                        padding = 'max_length',
                                        return_attention_mask = True)

    return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(),
            'labels': label }

In [31]:
#use pretrained tokenizer used from Roberta
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
ChatGPT_ds = ChatGPT_Dataset(train_path, tokenizer)
ChatGPT_ds.__getitem__(0)

{'input_ids': tensor([    0, 35581,  9178, 10780,  1992,    21,    10, 10780,  1992,  2034,
            11,     5,  1139,     9, 15516,  9178,     6,    15,     5,   230,
          4179, 10713,  2565,  5562,    11,   369,   580,  1156,     4,    20,
          1992,    21,  1357,    30,     5, 19766,  1671,  1885,     8,  9661,
         14977,    15,   262,   779, 41102,     4,    85,    21,  1367,     7,
          3670,    15,   195,   644,  9323,     6,     8,     7,  3057,    15,
           316,   392, 18202,     4,  1437, 50118, 50118,   133,  1992,   745,
            16,   122,    10,   940,  5238,     4,   345,    16,    10,   650,
          1280,     9,  1349,  1580,  2405,   583,     5,   745,     6,   341,
           855,    30,    10,   400,  7119,   265,     4,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [32]:
len(ChatGPT_ds)

300000

Create a data module to create datasets for training data set and its also going to return the data loaders.

In [33]:
class ChatGPT_Data_Module(pl.LightningDataModule):

  def __init__(self, train_path, batch_size = 16, max_token_len = 512, model_name = 'roberta-base'):
    super().__init__()
    self.train_path = train_path
    self.batch_size = batch_size
    self.max_token_len = max_token_len
    self.model_name = model_name
    self.tokenizer = RobertaTokenizer.from_pretrained(model_name)

  #create dataset (defined in previous class)
  def setup(self, stage = None):
    self.train_dataset = ChatGPT_Dataset(self.train_path, self.tokenizer)

  #return dataloader for training dataset
  #'num_workers' denote number of processes that generate batches in parallel
  #'shuffle = True' to randomly choose items
  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers = 2, shuffle = True)

In [34]:
chatgpt_data_module = ChatGPT_Data_Module(train_path)

In [35]:
chatgpt_data_module.setup()

In [36]:
dl = chatgpt_data_module.train_dataloader()

In [37]:
#return number of batches
len(dl)

#Output: 18750 since sample size is 300,000 after concatenating dataframes
#hence 300,000/16 = 18750

18750

# Creating the model

In [38]:
from transformers import RobertaConfig, RobertaModel, AdamW, get_cosine_schedule_with_warmup
import torch.nn as nn
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

In [39]:
class ChatGPT_Classifier(pl.LightningModule):

  #we can pass in batch of comments and would get back batch
  #roberta is a pretrained model that can be used for many downstream task
  #in our context we are using it for classification so we'd want to append a classification head onto the end of the model
  #to improve our performance even more, we will add a 2 layer neural network to the end (so a hidden layer --> final layer)
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.pretrained_model = RobertaModel.from_pretrained('roberta-base', return_dict = True)
    self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size) #hidden layer
    self.classification = nn.Linear(self.pretrained_model.config.hidden_size, self.config['n_labels']) #classification layer
    #torch can automatically initialise these layers, but using xavier_uniform
    #means we can initialise the weight of the NN layer ==> improving performance
    torch.nn.init.xavier_uniform(self.hidden.weight)
    torch.nn.init.xavier_uniform(self.classifier.weight)

# Evaluating the model